<a href="https://colab.research.google.com/github/vahidNaghshin/Data_structures_and_algorithms_in_Python/blob/main/Data_structures_and_algorithms_in_Python_Chapter_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Priority Queues - Chapter 9

In [ ]:
class PriorityQueueBase:
  class _Item:
    __slots__ = '_key', '_value'

    def __init__(self, k, v):
      self._key = k
      self._value = v
    
    def __lt__(self, other):
      return self._key < other._key
  
  def is_empty(self):
    return len(self)==0



In [ ]:
class _DoublyLinkedBase:
  class _Node:
    __slot__ = '_element', '_next', '_prev'
    def __init__(self, element, next, prev):
      self._element = element
      self._next = next
      self._prev = prev
  
  def __init__(self):
    self._header = self._Node(None, None, None)
    self._trailer = self._Node(None, None, None)
    self._header._next = self._trailer
    self._trailer._prev = self._header
    self._size = 0

  def __len__(self):
    return self._size
  
  def is_empty(self):
    return self._size == 0
  
  def _insert_between(self, e, predecessor, successor):
    
    new_node = self._Node(e, successor, predecessor)
    successor._prev = new_node
    predecessor._next = new_node
    self._size += 1
    
    return new_node

  def _delete_node(self, node):
    
    predecessor = node._prev
    successor = node._next
    successor._prev = predecessor
    predecessor._next = successor
    self._size -= 1
    element = node._element
    node._element, node._prev, node._next = None, None, None
    
    return element

In [ ]:
class PositionalList(_DoublyLinkedBase):
  class Position:
    def __init__(self, container, node):
      self._container = container
      self._node = node
    def element(self):
      return self._node._element
    def ــeqــ(self, other):
      return type(other) is type(self) and other.ـnode is self.ـnode
    def ــneــ(self, other):
      return not (self == other) 

  def _validate(self, p):
    if not isinstance(p, self.Position):
      raise TypeError( 'p must be proper Position type' ) 
    if p._container is not self:
      raise ValueError( 'p does not belong to this container' )
    if p._node._next is None: 
      raise ValueError( 'p is no longer valid' ) 
    return p._node

  def _make_position(self, node):
    if node is self._header or node is self._trailer:
      return None 
    else:
      return self.Position(self, node)
  
  def first(self):
    return self._make_position(self._header._next)

  def last(self):
    return self._make_position(self._trailer._prev)

  def before(self, p):
    node = self._validate(p)
    return self._make_position(node._prev)
  
  def after(self, p):
    node = self._validate(p)
    
    return self._make_position(node._next)

  def __iter__(self):
    cursor = self.first()
    while cursor is not None:
      yield cursor.element() 
      cursor = self.after(cursor)

  def _insert_between(self, e, predecessor, successor):
    node = super()._insert_between(e, predecessor, successor)
    return self._make_position(node)

  def add_first(self, e): 
    return self._insert_between(e, self._header, self._header._next)
  
  def add_last(self, e):
    return self._insert_between(e, self._trailer._prev, self._trailer)

  def add_before(self, p, e):
    original = self._validate(p)
    return self._insert_between(e, original._prev, original)

  def add_after(self, p, e):
    original = self._validate(p)
    return self._insert_between(e, original, original._next)
  
  def delete(self, p):
    original = self._validate(p)
    return self._delete_node(original) 


  def replace(self, p, e):
    original = self._validate(p)
    old_value = original._element
    original._element = e 
    return old_value


In [ ]:
class Empty(Exception):
  pass

In [ ]:
class UnsortedPriorityQueue(PriorityQueueBase):
  def _find_min(self):
    if self.is_empty():
      raise Empty('Priority queue is empty')
    small = self._data.first()
    walk = self._data.after(small)
    while walk is not None:
      if walk.element() < small.element(): 
        small = walk
      walk = self._data.after(walk)
    return small
  
  def __init__(self):
    self._data = PositionalList()
  
  def __len__(self):
    return len(self._data)
  
  def add(self, key, value):
    self._data.add_last(self._Item(key, value))
  
  def min(self):
    p = self._find_min()
    item = p.element()
    return (item._key, item._value)
  
  def remove_min(self):
    p = self._find_min()
    item = self._data.delete(p)
    return (item._key, item._value)
  

In [ ]:
class SortedPriorityQueue(PriorityQueueBase):
  def __init__(self):
    self._data = PositionalList()
  
  def __len__(self):
    return len(self._data)
  
  def add(self, key, value):
    newest = self._Item(key, value)
    walk = self._data.last()
    while walk is not None and newest < walk.element():
      walk = self._data.before(walk)
    if walk is None:
      self._data.add_first(newest)
    else:
      self._data.add_after(walk, newest)
  
  def min(self):
    if self.is_empty():
      raise Empty('Priority queue is empty.')
    p = self._data.first()
    item = p.element()
    return (item._key, item._value)
  
  def remove_min(self):
    if self.is_empty():
      raise Empty( 'Priority queue is empty.' ) 
    item = self._data.delete(self._data.first()) 
    return (item._key, item._value)

In [ ]:
class HeapPriorityQueue(PriorityQueueBase):
  """A min-oriented priority queue implemented with a binary heap."""
  def _parent(self, j):
    return (j-1)//2
    
  def _left(self, j):
    return 2*j+1 

  def _right(self, j):
    return 2*j+2 
  
  def _has_left(self, j):
    return self._left(j) < len(self._data) 
  
  def _has_right(self, j):
    return self._right(j) < len(self._data)
  
  def _swap(self, i, j):
    self._data[i], self._data[j] = self._data[j], self._data[i]
  
  def _upheap(self, j):
    parent = self._parent(j)
    if j > 0 and self._data[j] < self._data[parent]:
      self._swap(j, parent) 
      self._upheap(parent)
  
  def _downheap(self, j): 
    if self._has_left(j):
      left = self._left(j) 
      small_child = left
      if self._has_right(j):
        right = self._right(j)
        if self._data[right] < self._data[left]:
          small_child = right
      if self._data[small_child] < self._data[j]:
        self._swap(j, small_child)
        self._downheap(small_child)

  #------------- public behaviours
  # def __init__ (self):
  #   self._data = []

  def __init__ (self, contents=()):
    self._data = [self._Item(k,v) for k, v in contents]
    if len(self._data) > 1:
       self._heapify()
    
  def _heapify(self):
    start = self._parent(len(self) - 1) 
    for j in range(start, -1, -1):
      self._downheap(j)
    
  
  def __len__ (self):
    return len(self._data)
  
  def add(self, key, value):
    self._data.append(self._Item(key, value))
    self._upheap(len(self._data) - 1)

  def min(self):
    if self.is_empty():
      raise Empty( 'Priority queue is empty.' ) 
      item = self._data[0]
      return (item._key, item._value)

  def remove_min(self):
    if self.is_empty():
      raise Empty( 'Priority queue is empty.' )
    self._swap(0, len(self._data) - 1) 
    item = self._data.pop()
    self._downheap(0)
    return (item._key, item._value)    
  

In [ ]:
from heapq import *
L = []
heappush(L, 2)
heappush(L, 1)
heappush(L, 20)

In [ ]:
heappop(L)

1

In [ ]:
heappushpop(L, 4)

2

In [ ]:
heapreplace(L, 1)

4

In [ ]:
heapify([3,2,5,1,0])

In [ ]:
def pq_sort(C):
  n = len(C)
  P = PriorityQueue()
  for j in range(n):
    element = C.delete(C.first())
    P.add(element, element) 
  for j in range(n):
    (k,v) = P.remove_min() 
    C.add_last(v)

In [ ]:
class AdaptableHeapPriorityQueue(HeapPriorityQueue):
  class Locator(HeapPriorityQueue._Item):
    __slots__ = '_index'
    def __init__(self, k, v, j):
      super.__init__(k, v)
      self._index = j
  
  def _swap(self, i, j):
    super()._swap(i,j) 
    self._data[i]._index = i 
    self._data[j]._index = j
  

  def _bubble(self, j):
    if j > 0 and self._data[j] < self._data[self._parent(j)]:
      self._upheap(j) 
    else:
      self._downheap(j)
  
  def add(self, key, value):
    token = self.Locator(key, value, len(self. data)) # initiaize locator index 
    self._data.append(token)
    self._upheap(len(self._data) - 1)
    return token
  
  def update(self, loc, newkey, newval):
    j = loc._index
    if not (0 <= j < len(self) and self._data[j] is loc):
      raise ValueError( 'Invalid locator' ) 
      loc._key = newkey
    loc._value = newval
    self._bubble(j)

  def remove(self, loc):
    j = loc._index
    if not (0 <= j < len(self) and self._data[j] is loc):
      raise ValueError( 'Invalid locator' )
    if j == len(self) - 1: 
      self._data.pop()
    else:
      self._swap(j, len(self) - 1) 
      self._data.pop( )
      self._bubble(j)

    return (loc. key, loc. value)

#  Reinforcement

In [ ]:
#@title R-9.4
Q = SortedPriorityQueue()

#add an event with timestamp 2
timestamp = 2
flight_id = 3
Q.add(timestamp, flight_id)

#add an event with timestamp .3
timestamp = .3
flight_id = 5
Q.add(timestamp, flight_id)

#add an event with timestamp 5
timestamp = 5
flight_id = 6
Q.add(timestamp, flight_id)
print(len(Q))
print(Q.remove_min())
print(Q.remove_min())
print(Q.remove_min())

3
(0.3, 5)
(2, 3)
(5, 6)


In [ ]:
#@title R-9.5
#the same modification in SortedPriorityQueue shoud be applied.

In [ ]:
#@title R-9.12
# we can use the minus key for the traditional minimum-oriented sorted key priority key.

# Creativity

In [ ]:
#@title C-9.26

class Stack(SortedPriorityQueue):
  """A slack ADT developed by the Sorted priority queue"""
  def __init__(self):
    #initialize the key of the node to 0
    #reduce the value of the key of each node by one so the LIFO structure is preserved
    super.__init__()
    self._key = 0
  
  def push(self, e):
    self.add(self._key, e)
    #reduce the key of the next added node by one so, it will be prioritised to the previously added one
    self._key -= 1
  
  def pop(self):
    self.remove_min()
  
  def top(self):
    return self.min()

In [ ]:
#@title C-9.27

class Queue(SortedPriorityQueue):
  """A Queue ADT developed by the Sorted priority queue"""
  def __init__(self):
    #initialize the key of the node to 0
    #increase the value of the key of each node by one so the FIFO structure is preserved
    super.__init__()
    self._key = 0
  
  def enqueue(self, e):
    self.add(self._key, e)
    #reduce the key of the next added node by one so, it will be prioritised to the previously added one
    self._key += 1
  
  def dequeue(self):
    self.remove_min()
  
  def first(self):
    return self.min()



In [ ]:
#@title C-9.29

class SortedPriorityQueue(PriorityQueueBase):
  """Implementing the sorted priority queue using the Python list"""

  def __init__(self):
    self._data = []
  
  def __len__(self):
    return len(self._data)
  
  def add(self, key, value):
    newest = self._Item(key, value)
    
    if not self.is_empty():
      c = 0
      new_data = (len(self) + 1) * [None]
      
      while 0 <= c < len(self) and newest > self._data[c]:
    
        walk = self._data[c]
        new_data[c] = walk
        c += 1
        
      
      new_data[c] = newest
      new_data[c+1:] = self._data[c:]
      self._data = new_data
    else:
      self._data.append(newest)

  def min(self):
    if self.is_empty():
      raise Empty('Priority queue is empty.')
    item = self._data[0]
    
    return (item._key, item._value)
  
  def remove_min(self):
    if self.is_empty():
      raise Empty( 'Priority queue is empty.' ) 
    item = self._data[0]
    del self._data[0]
    return (item._key, item._value)

In [ ]:
#@title C-9.30

def _upheap(self, j):
    """non-recursive implementation of the upheap method"""
    parent = self._parent(j)
    while j > 0 and self._data[j] < self._data[parent]:
      self._swap(j, parent)
      j = parent 
      parent = self._parent(j)

In [ ]:
#@title C-9.31
def _downheap(self, j):
    flag = True # in case the current position does not violate the heap order
    while flag and j < len(self) and self._has_left(j):
      left = self._left(j)
      small_child = left
      
      if self._has_right(j):
        right = self._right(j)
        if self._data[right] < self._data[left]:
          small_child = right
      
      if self._data[small_child] < self._data[j]:
        self._swap(j, small_child)
        j = small_child

        parent = self._parent(j)
        if self._has_left(parent) and self._has_right(parent):
          right, left = self._right(parent), self._left(parent)
          
          if self._data[right] < self._data[left]:
            self._swap(left, right)
      else:
        flag = False

In [ ]:
#@title C-9.32